Задача 1. Стек

In [3]:
class Stack:
    '''Stack object.'''

    def __init__(self) -> None:
        self.items = []

    def push(self, item: str) -> None:
        self.items.insert(0, item)

    def pop(self) -> 'Stack':
        return self.items.pop(0)

    def peek(self) -> str:
        return self.items[0]

In [6]:
stack = Stack()
stack.push('1')
stack.push('2')
stack.push('3')
stack.peek()

'3'

In [8]:
stack.pop()
stack.peek()

'2'

In [49]:
class TaskManager:
    '''Task manager.'''

    def __init__(self) -> None:
        self.stack = Stack()
        self.tasks = []

    def new_task(self, task: str, priority: int) -> None:
        self.stack.push((task, priority))
        self.tasks.append(self.stack.pop())

    def get_tasks(self) -> dict:
        res = dict((k, v) for k, v in self.tasks)
        res = {k: v for k, v in sorted(res.items(), key=lambda item: item[1])}
        return res

In [50]:
manager = TaskManager()
manager.new_task("сделать уборку", 4)
manager.new_task("помыть посуду", 4)
manager.new_task("отдохнуть", 1)
manager.new_task("поесть", 2)
manager.new_task("сдать дз", 2)
manager.get_tasks()

{'отдохнуть': 1,
 'поесть': 2,
 'сдать дз': 2,
 'сделать уборку': 4,
 'помыть посуду': 4}

Задача 2. Кэширование запросов

In [2]:
from collections import OrderedDict

class LRUCache:
    '''LRU Cache.'''

    def __init__(self, capacity: int) -> None:
        self.__cache = OrderedDict()
        self.capacity = capacity

    @property
    def cache(self) -> str:
        key = next(reversed(self.__cache))
        return self.__cache[key]        

    @cache.setter
    def cache(self, new_elem: tuple) -> None:
        key, value = new_elem[0], new_elem[1]
        self.__cache[key] = value
        self.__cache.move_to_end(key)
        if len(self.__cache) > self.capacity:
            self.__cache.popitem(last=False)

    def get(self, key: str) -> str:
        if key not in self.__cache:
            raise KeyError
        else:
            self.__cache.move_to_end(key)
            return self.__cache[key]

    def print_cache(self) -> None:
        print(self.__cache)    

In [4]:
# Создаём экземпляр класса LRU Cache с capacity = 3
cache = LRUCache(3)

# Добавляем элементы в кэш
cache.cache = ("key1", "value1")
cache.cache = ("key2", "value2")
cache.cache = ("key3", "value3")

# Выводим текущий кэш
cache.print_cache() # key1 : value1, key2 : value2, key3 : value3


# Получаем значение по ключу
print(cache.get("key2")) # value2


# Добавляем новый элемент, превышающий лимит capacity
cache.cache = ("key4", "value4")


# Выводим обновлённый кэш
cache.print_cache() # key2 : value2, key3 : value3, key4 : value4


# Ожидаемый вывод в консоли:

# LRU Cache:
# key1 : value1
# key2 : value2
# key3 : value3

# value2

# LRU Cache:
# key3 : value3
# key2 : value2
# key4 : value4

OrderedDict([('key1', 'value1'), ('key2', 'value2'), ('key3', 'value3')])
value2
OrderedDict([('key3', 'value3'), ('key2', 'value2'), ('key4', 'value4')])


Задача 3. Кэширование для ускорения вычислений

In [9]:
class CachingFunction:
    '''Caching decorator.'''
    
    def __init__(self, func) -> None:
        self.func = func
        self.__cache = {}
        self.__count = 1

    def __call__(self, *args, **kwargs) -> 'CachingFunction':
        key = self.func(*args, **kwargs)

        if self.__cache.get(key) is None:
            self.__cache[key] = self.__count
        else:
            self.__cache[key] += 1

        if self.__cache[key] > 1:
            key_list = list(self.__cache.keys())          # |
            val_list = list(self.__cache.values())        # | Time complexity: O(1)
            position = val_list.index(self.__cache[key])  # | Auxiliary space: O(1)
            key = key_list[position]                      # |
            return key
        return key


@CachingFunction
def fib(n: int) -> int:
    if n < 2:
        return n
    else:
        return fib(n-1) + fib(n-2)


for i in range(1, 11):
    print(fib(i), end=' ')

1 1 2 3 5 8 13 21 34 55 

Задача 4. Крестики нолики

In [9]:
import numpy as np

BOARD_ROWS = 3
BOARD_COLS = 3

In [10]:
class Player:
    '''Player class.'''

    def __init__(self, name):
        self.name = name

    def make_move(self, available_moves):
        print("{}'s turn".format(self.name))
        while True:
            try:
                row = int(input('Enter the row: '))
                col = int(input('Enter the col: '))
            except:
                continue
            action = (row, col)
            if action in available_moves:
                return action

In [11]:
class Game:
    '''Game class.'''

    def __init__(self, player1, player2):
        self.player1 = player1
        self.player2 = player2
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.gameover = False
        self.turn = 1

    def play(self):
        while not self.gameover:
            self.show_board()
            moves = self.available_moves()
            p1_action = self.player1.make_move(moves)
            self.update_board(p1_action)
            winner = self.check_win()
            if self.gameover:
                if winner == 1:
                    print(self.player1.name, 'wins!')
                else:
                    print('tie!')
                self.reset()
                break
            else:
                self.show_board()
                moves = self.available_moves()
                p2_action = self.player2.make_move(moves)
                self.update_board(p2_action)
                winner = self.check_win()
                if self.gameover:
                    if winner == -1:
                        print(self.player2.name, 'wins!')
                    else:
                        print('tie!')
                    self.reset()
                    break
        replay = input('Do you want to play again? (y/n): ')
        if replay == 'y':
            self.play()

    def show_board(self):
        # p1: X and p2: 0
        print('')
        for i in range(0, BOARD_ROWS):
            print('-------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'X'
                if self.board[i, j] == -1:
                    token = '0'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('-------------')

    def available_moves(self):
        moves = []
        for i in range(0, BOARD_ROWS):
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 0:
                    moves.append((i, j))
        return moves
    
    def update_board(self, move):
        self.board[move] = self.turn
        # check player
        self.turn = -1 * self.turn

    def check_win(self):
        # row
        row_sum = self.board.sum(axis=1)
        for row_val in row_sum:
            if row_val == 3 or row_val == -3:
                self.gameover = True
                return -1 * self.turn
        # col
        col_sum = self.board.sum(axis=0)
        for col_val in col_sum:
            if col_val == 3 or col_val == -3:
                self.gameover = True
                return -1 * self.turn
        # diagonals
        diag1_sum = self.board.trace()
        diag2_sum = np.fliplr(self.board).trace()

        if diag1_sum == 3 or diag2_sum == 3:
            self.gameover = True
            return 1
        elif diag1_sum == -3 or diag2_sum == -3:
            return -1
        
        # tie
        # no available moves
        if len(self.available_moves()) == 0:
            self.gameover = True
            return 0
    
    def reset(self):
        self.gameover = False
        self.turn = 1
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))

In [12]:
if __name__ == '__main__':
    name1 = input('Enter player 1 name: ')
    name2 = input('Enter player 2 name: ')
    p1 = Player(name1)
    p2 = Player(name2)
    g = Game(p1, p2)
    g.play()


-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
Elon's turn

-------------
|   |   |   | 
-------------
|   |   | X | 
-------------
|   |   |   | 
-------------
Mark's turn

-------------
| 0 |   |   | 
-------------
|   |   | X | 
-------------
|   |   |   | 
-------------
Elon's turn

-------------
| 0 |   |   | 
-------------
|   | X | X | 
-------------
|   |   |   | 
-------------
Mark's turn

-------------
| 0 |   |   | 
-------------
|   | X | X | 
-------------
|   |   | 0 | 
-------------
Elon's turn
Elon wins!
